In [ ]:
# mount Google Drive if required, otherwise skip

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-12-07 00:52:38--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-07 00:52:39--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-07 00:52:40--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:44

In [ ]:
# nlpaug
! pip install nlpaug
! pip install transformers
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import nlpaug
import nlpaug.augmenter.word as naw
import json

import os
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import sparknlp
from tqdm.notebook import tqdm

# change data source if not utilizing Google Colab
data_source = "/content/drive/MyDrive"


spark = sparknlp.start(gpu=True)

# Load in corpus and create spark dataframe.  Using text as dictionary key will 
# eliminate exact duplicates.

corpus = open(f"{data_source}/SIC-annotated.jsonl").read()
corpus_raw = [json.loads(line) for line in tqdm(corpus.splitlines())]

#for line in tqdm(corpus_raw):
    #SIC_dict[line["provision"]] = line["source"][:2]

count_dict = {}
SIC_dict = {}

for line in tqdm(corpus_raw):
    SIC_dict[line["provision"]] = line["source"][:2]

SIC_columns = ["text", "code"]
    
SIC_df = spark.createDataFrame(SIC_dict.items(), schema=SIC_columns)

# get code counts, maximum count value, and create spark dataframe from grouped data
count_df = SIC_df.groupBy("code").count().toPandas()
max_value= count_df["count"].max()
count_spark = spark.createDataFrame(count_df)

spark_counts = count_spark.collect()

for row in spark_counts:
  count_dict[row["code"]] = int(max_value / row["count"])

print(count_dict)

SIC_collect = SIC_df.collect()

#print(count_dict)

SIC_list = []

for line in tqdm(SIC_collect):
  
  industry = line["code"]

  if count_dict[industry] > 1:

    SIC_list.append([line["text"], industry])

    aug = naw.SynonymAug(aug_src="wordnet", aug_max=25)

    augmented = aug.augment(line["text"], n=count_dict[industry])

    for augment in augmented:
      SIC_list.append([augment, industry])

  else:
    SIC_list.append([line["text"], industry])

#print(len(SIC_dict))

with open(f"{data_source}/SIC-aug-list.jsonl", "w") as fp:
  #for k in SIC_dict.keys():
  for entry in SIC_list:
    #fp.write(json.dumps({"text": k, "code": SIC_dict[k]}) + "\n")
    fp.write(json.dumps({"text": entry[0], "code": entry[1]}) + "\n")

fp.close()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


  0%|          | 0/1850284 [00:00<?, ?it/s]

  0%|          | 0/1850284 [00:00<?, ?it/s]

{'07': 103, '51': 9, '54': 42, '15': 20, '29': 25, '42': 47, '73': 1, '87': 7, '64': 62, '30': 17, '34': 13, '59': 8, '01': 107, '28': 1, '22': 118, '35': 3, '52': 65, '16': 53, '47': 28, '99': 1492, '31': 56, '70': 11, '27': 28, '61': 5, '75': 48, '17': 47, '26': 25, '09': 669, '46': 24, '78': 35, '60': 4, '23': 27, '41': 4244, '55': 20, '08': 780, '38': 2, '40': 92, '25': 37, '02': 1318, '44': 20, '82': 27, '53': 21, '58': 9, '81': 370, '33': 15, '48': 4, '67': 1, '79': 24, '24': 47, '32': 54, '20': 4, '56': 17, '36': 2, '10': 18, '37': 6, '49': 3, '63': 6, '65': 7, '39': 27, '62': 6, '12': 21, '83': 458, '13': 2, '21': 78, '14': 30, '72': 31, '76': 306, '80': 6, '50': 7, '45': 20, '57': 44}


  0%|          | 0/1081177 [00:00<?, ?it/s]